In [1]:
import pandas as pd
import warnings
import numpy as np

warnings.filterwarnings("ignore")
np.set_printoptions(linewidth=1000000)

pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [2]:
import pyodbc
cnxn = pyodbc.connect("DRIVER={SQL Server Native Client 11.0};"
                      "SERVER=10.22.2.62\VHDVTT;"
                      "DATABASE=VNPAYEWT1;"
                      "Trusted_Connection=yes;")

vastxn = pd.read_sql_query("select * from  [VNPAYEWT1].[dbo].[ALLTIME_VAS_TXN]", cnxn)



In [4]:
# Loc ra nhung TXN thanh cong của SDK và GD bình thường:
sdk_txn = vastxn.loc[(vastxn['STATUS'] == '2') & (vastxn['TXN_TYPE'] == 22), :]
nonsdk_txn = vastxn.loc[(vastxn['STATUS'] == '4') & (vastxn['TXN_TYPE'] != 22), :]
vastxn_tc = pd.concat([sdk_txn, nonsdk_txn])


# RFM FOR VAS_TXN ALLTIME


### Tinh Segment cho tung quy

In [6]:
seg_map = {r'555': 'Champions',
           r'[3-5][4-5][1-5]': 'Loyal Customers', # Điều chỉnh monetary 1-5
           r'[1-2][4-5][1-5]': 'Losing Loyal Customers', # đối với trường hợp 1-2
           r'[4-5][2-3][2-3]': 'Promissing',
           r'411': 'Warm Leads', 
           r'311': 'Cold Leads',
           r'[1-2][1-3][4-5]': "Losing Big spenders", # => chuyển từ "Shouldn't lose" => "Losing Big spenders"
           r'[1-2]3[3-4]': "Sleepers",
           r'[3-5][1-3][4-5]': 'Big Spender', #=> thêm phần này: là những khách hàng chịu chi tiêu nhiều
           r'[1-2][1-2][1-2]': 'Lost',
           r'[4-5][1-3][1-3]': 'Recent Customers', # đối với các khách hàng có recency cao => mới  giao dịch => khách hàng mới
           r'[1-3][1-3][1-3]': 'Need Attention'} # là các trường hợp có điểm 3 trong 3 chữ số


In [7]:
pd.to_datetime(vastxn_tc['CREATED_DATE'].dt.date.min())

Timestamp('2020-04-01 00:00:00')

In [8]:
# Mình sẽ lấy các mốc thời gian kết thúc của quý và tính segment ngược về trước.

# Helper function này sẽ lấy ngày cuối cùng từng quý trong các năm của data
def get_eoq(dataframe):
    '''
    Function để lấy ra những ngày cuối cùng của mỗi quý giao dịch. 
    Thông tin lấy từ trường CREATED_DATE trong bảng vas_txn

    Return là 1 list timestamp những ngày cuối cùng của mỗi quý giao dịch
    '''
    years = list(dataframe['CREATED_DATE'].dt.year.unique())
    years.sort()
    eoq_dates = ['-03-31', '-06-30', '-09-30', '-12-31']
    eoq_list = []
    eoq_list.append(pd.to_datetime(dataframe['CREATED_DATE'].dt.date.min()))
    for year in years:
        for eoq_date in eoq_dates:
            eoq = pd.to_datetime(str(year) + eoq_date)
            if eoq < dataframe['CREATED_DATE'].max() and (dataframe['CREATED_DATE'] <= eoq).sum() > 0:
                eoq_list.append(pd.to_datetime(str(year) + eoq_date))
    return eoq_list

eoq_dates = get_eoq(vastxn_tc)
eoq_dates

[Timestamp('2020-04-01 00:00:00'),
 Timestamp('2020-06-30 00:00:00'),
 Timestamp('2020-09-30 00:00:00'),
 Timestamp('2020-12-31 00:00:00'),
 Timestamp('2021-03-31 00:00:00'),
 Timestamp('2021-06-30 00:00:00'),
 Timestamp('2021-09-30 00:00:00'),
 Timestamp('2021-12-31 00:00:00'),
 Timestamp('2022-03-31 00:00:00')]

In [9]:
def rfm_generator(txn_df):
    '''
    Function này dùng để tính toán các chỉ số RFM và mapping các segment cho các chỉ số đấy.
    Đầu vào là dataframe chứa các GD thành công, với yêu cầu phải có các trường:
        + CREATED_DATE  - [datetime]    - Thời điểm phát sinh GD thành công
        + TXN_ID        - [any format]  - ID của GD
        + AMOUNT        - [int]         - Số tiền GD
        + USER_MAP_ID   - [string]      - ID của KH
    
    Return: Dataframe chứa các trường: USER_MAP_ID, SEGMENT, R_SCORE, F_SCORE, M_SCORE
    '''
    # Define threshold:
    r_bins = [-1, 16, 34, 54, 72, 100000000]
    f_bins = [-1, 3, 8, 18, 38, 100000000]
    m_bins = [-1, 664375, 2766000, 6610000, 12106000, 10000000000000]
    
    # Get max txn date:
    max_txn_date = txn_df['CREATED_DATE'].max()

    # Calculating RFM
    rfm_q = txn_df.groupby('USER_MAP_ID').agg({
        'CREATED_DATE': lambda thoi_gian_gd: (max_txn_date - thoi_gian_gd.max()).days,    # Calculating recency
        'TXN_ID': 'count',                                                             # Calculating frequency
        'AMOUNT': lambda sotienthanhtoan: round(sotienthanhtoan.mean(), 0)  # Calculating monetary    
    }).reset_index()
    # Rename columns:
    rfm_q.columns = ['USER_MAP_ID', 'recency', 'frequency', 'monetary']

    ###### SCORING:
    # Scoring recency:
    rfm_q['R_SCORE'] = pd.cut(rfm_q['recency'], r_bins ,labels=[5, 4, 3, 2, 1])

    # Scoring frequency:
    rfm_q['F_SCORE'] = pd.cut(rfm_q['frequency'], f_bins ,labels=[1, 2, 3, 4, 5])

    # Scoring monetary:
    rfm_q['M_SCORE'] = pd.cut(rfm_q['monetary'], m_bins ,labels=[1, 2, 3, 4, 5])

    ###### SEGMENTATION MAPPING 
    rfm_q['rfm_score'] = rfm_q.R_SCORE.astype('str') + rfm_q.F_SCORE.astype('str') + rfm_q.M_SCORE.astype('str')
    rfm_q['SEGMENT'] = rfm_q['rfm_score'].replace(seg_map, regex=True)

    return rfm_q



In [10]:
# Lặp qua các quý => cắt dữ liệu GD của từng quý => tính toán RFM và segment 
# => lưu kết quả tính toán vào trong 1 dictionary
quater_seg_dict = {}
for i, eoq in enumerate(eoq_dates):
    try:
        txn_until_quater = vastxn_tc.loc[(vastxn_tc['CREATED_DATE'] >= eoq) & (vastxn_tc['CREATED_DATE'] < eoq_dates[i+1]), :]
        quater = str(eoq_dates[i+1].to_period('Q'))
        # Calculating RFM
        quater_seg_dict[quater] = rfm_generator(txn_until_quater)[['USER_MAP_ID', 'SEGMENT', 'R_SCORE', 'F_SCORE', 'M_SCORE']]
        print(i, quater, 'done')
    except:
        maxdate = pd.to_datetime(vastxn_tc['CREATED_DATE'].max().date())
        fromdate = pd.to_datetime(maxdate) - pd.Timedelta(90, 'D')
        txn90days = vastxn_tc.loc[(vastxn_tc['CREATED_DATE'] >= fromdate) & (vastxn_tc['CREATED_DATE'] <= maxdate), :]
        quater = 'Q-UPDATED'
        quater_seg_dict[quater] = rfm_generator(txn90days)[['USER_MAP_ID', 'SEGMENT', 'R_SCORE', 'F_SCORE', 'M_SCORE']]
        print(i, quater, 'done')


0 2020Q2 done
1 2020Q3 done
2 2020Q4 done
3 2021Q1 done
4 2021Q2 done
5 2021Q3 done
6 2021Q4 done
7 2022Q1 done
8 Q-UPDATED done


In [11]:
def get_no_txn(db1, db2):
    '''
    Function này giúp lấy ra những user_map_id có giao dịch trong db1 nhưng không có
    GD trong db2, nếu không GD thì sẽ được fill là "No TXN". 
    Với:
        + db1['USER_MAP_ID', 'SEGMENT']
        + db2['USER_MAP_ID', 'SEGMENT']

    Return là 1 dataframe có có 2 field: ['USER_MAP_ID', 'SEGMENT'], với: 
        + User_map_id: là những id có trong db1 nhưng không có trong db2
        + Segment = "No TXN"
    '''
    df = pd.merge(
        left = db1,
        right = db2,
        on=['USER_MAP_ID'],
        how="outer",
        indicator=True
    )
    df = df[df['_merge']=='left_only']
    df.drop(columns = ['SEGMENT_x', '_merge', 'R_SCORE_x', 'F_SCORE_x', 'M_SCORE_x'], inplace=True)
    df.columns = ['USER_MAP_ID', 'SEGMENT', 'R_SCORE', 'F_SCORE', 'M_SCORE']
    df['SEGMENT'] = df['SEGMENT'].fillna('No TXN')
    
    return df

In [12]:
# Rà xoát những trường hợp có GD ở quý này nhưng không có GD ở những quý sau đó:
quater_list = list(quater_seg_dict.keys())
for i, quater1 in enumerate(quater_list):
    for quater2 in quater_list[(i+1):]:
        # Lấy những user_map_id có GD trong quý trước những k có gd trong những quý sau:
        no_txn_df = get_no_txn(quater_seg_dict[quater1], quater_seg_dict[quater2])

        # Nối những user_map_id này vào trong dataframe của các quý sau đó:
        quater_seg_dict[quater2] = pd.concat([quater_seg_dict[quater2], no_txn_df], axis=0)
        

In [13]:
# Nối các bảng segment của từng quý theo chiều dọc
segment_db_list = list(quater_seg_dict.values())
cus_seg = pd.concat(segment_db_list, keys = quater_list, axis = 0).reset_index()
# Drop cột level 1
cus_seg.drop(columns='level_1', inplace=True)
# Đổi tên level_0 => Quater
cus_seg.rename(columns = {'level_0': 'QUATER'}, inplace=True)

# Output dataset

In [111]:
cus_seg.to_csv('./Export dataset/CUSTOMER_SEGMENT_220415.csv', index = False)
